In [1]:
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [2]:
import sys
import asyncio

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__


## Local MCP server

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient


# 여러 서버를 다룰 수 있는 mcp 클라이언트를 생성
client = MultiServerMCPClient(
    {
        "local_server": { # resource에 있는 서버의 별칭
                "transport": "stdio", # 표준 입출력으로 MCP와 통신, 별도 네트워크 포트가 아니라 프로세스간 IO로 연결, stdio 아니면 http 중 하나일 것이다.
                "command": "python", # 서버를 실행할 명령어
                "args": ["resources/2.1_mcp_server.py"], # 실행할 스크립트 경로
            } # 실행과 동시에 서버가 실행될 것 이다.
    }
)

In [4]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [5]:
prompt

'\n    당신은 Langchain, LangGraph, LangSmith에 대한 질문에 답변하는 친절한 어시스턴트 입니다.\n\n    질문에 답변하기 위해 다음 tools/resources를 사용할 수 있습니다.\n    - search_web: Search the web for information\n    - github_file: Access the langchain-ai repo files\n    \n    사용자가 LangChain, LangGraph or LangSmith와 관련 없는 질문을 하면, "죄송합니다. 저는 랭체인, 랭그레프, 랭스미스에 대해서만 답변할수 있어요." 라고 답변합니다.\n    \n    당신은 사용자 질문에 답변을 위해 다수의 tool과 resouce를 사용할 수 있습니다.\n    \n    당신은 사용자의 질문을 보다 잘 이해하기 위해 사용자에게 질문의 명확하게 식별하기 위한 질문을 할수 있습니다.(reasking)\n    '

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
    system_prompt=prompt
)

In [7]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="langchain-mcp-adapters에 대해서 알려줘.")]},
    config=config
)

In [8]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='langchain-mcp-adapters에 대해서 알려줘.', additional_kwargs={}, response_metadata={}, id='22bc9269-c318-4a90-a7d3-acb382e1e4e9'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 156, 'prompt_tokens': 319, 'total_tokens': 475, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D5KrkTmEFGDtEvlF8TRaC2gtbiUcO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c2602-6bef-77a2-af36-5cfc88a5c2cc-0', tool_calls=[{'name': 'search_web', 'args': {'query': 'langchain-mcp-adapters'}, 'id': 'call_jdyJEPgMVne0jLBpdmP2kXdb', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'i

## Online MCP

In [9]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [10]:
agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
)

In [ ]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke( # tools = await client.get_tools() 때문에 비동기로 호출하는 것이 안전
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='c82e3556-f96b-4773-a4af-d79358cf5f69'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 155, 'prompt_tokens': 296, 'total_tokens': 451, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D5L2PwqyarhkSto6x7kaWEpn67D7Q', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c260c-8308-7ad0-abd4-d9ed5f14ea21-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'America/New_York'}, 'id': 'call_6nc682yMR5BenI0DskcDxV4B', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens':